In [1]:
# 프로젝트 디렉토리에서 실행.
# jupyter notebook에서 django shell을 사용할수 있도록 환경변수 설정.

In [1]:
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'config.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

import django
django.setup()

In [7]:
from polls.models import Question, Choice, SubChoice

# 전체조회
- all()

In [3]:
# Model클래스이름.objects.XXXXXXX
qs = Question.objects.all()
qs

<QuerySet [<Question: 좋아하는 색깔은 무엇입니까?>, <Question: 좋아하는 동물은 무엇입니까?>, <Question: 가고 싶은 나라를 선택하세요>]>

In [10]:
qs = SubChoice.objects.all()
qs

<QuerySet [<SubChoice: 검은색이다>, <SubChoice: 파랑색이다>, <SubChoice: 빨강색이다>, <SubChoice: 우리나라다>, <SubChoice: 가고싶은나라다>, <SubChoice: 신분세탁국가다>, <SubChoice: 강아지는 귀엽다>, <SubChoice: 고양이는 귀엽다>, <SubChoice: 거북이는 느리다>, <SubChoice: 보드카의나라다>, <SubChoice: 놀러가고싶은나라다>]>

In [26]:
# QuerySet.query : sql문 조회
print(qs.query)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question"


In [29]:
for q in qs:
    print(q,type(q))

좋아하는 색깔은 무엇입니까? <class 'polls.models.Question'>
좋아하는 동물은 무엇입니까? <class 'polls.models.Question'>
가고 싶은 나라를 선택하세요 <class 'polls.models.Question'>


In [31]:
# 특정 조회 결과 하나 조회 - indexing
# 범위 조회: slicing / 음수 인덱스는 사용불가.
qs[0], qs[1:]

(<Question: 좋아하는 색깔은 무엇입니까?>,
 [<Question: 좋아하는 동물은 무엇입니까?>, <Question: 가고 싶은 나라를 선택하세요>])

In [33]:
qs.last(), qs.first()

(<Question: 가고 싶은 나라를 선택하세요>, <Question: 좋아하는 색깔은 무엇입니까?>)

In [34]:
qs = Choice.objects.all()
qs

<QuerySet [<Choice: 검정색>, <Choice: 파랑색>, <Choice: 빨강색>, <Choice: 한국>, <Choice: 미국>, <Choice: 영국>, <Choice: 강아지>, <Choice: 고양이>, <Choice: 거북이>, <Choice: 러시아>, <Choice: 오스트레일리아>]>

In [35]:
qs.order_by('choice_text') # asc

<QuerySet [<Choice: 강아지>, <Choice: 거북이>, <Choice: 검정색>, <Choice: 고양이>, <Choice: 러시아>, <Choice: 미국>, <Choice: 빨강색>, <Choice: 영국>, <Choice: 오스트레일리아>, <Choice: 파랑색>, <Choice: 한국>]>

In [36]:
qs.order_by('-choice_text') # dsc

<QuerySet [<Choice: 한국>, <Choice: 파랑색>, <Choice: 오스트레일리아>, <Choice: 영국>, <Choice: 빨강색>, <Choice: 미국>, <Choice: 러시아>, <Choice: 고양이>, <Choice: 검정색>, <Choice: 거북이>, <Choice: 강아지>]>

In [28]:
qs = Choice.objects.all().order_by('id')
qs

<QuerySet [<Choice: 검정색>, <Choice: 파랑색>, <Choice: 빨강색>, <Choice: 한국>, <Choice: 미국>, <Choice: 영국>, <Choice: 강아지>, <Choice: 고양이>, <Choice: 거북이>, <Choice: 러시아>, <Choice: 스페인>]>

In [21]:
print(qs.query)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."vote", "polls_choice"."question_id" FROM "polls_choice" ORDER BY "polls_choice"."choice_text" DESC


In [41]:
c = qs.first()
print(type(c))

<class 'polls.models.Choice'>


In [43]:
# 조회한 Model의 속성을 조회 -> 특정 컬럼값 조회
# Choice: id(pk), choice_text, vote, question_id
c.id, c.choice_text, c.vote, c.question, c.question_id

(4, '한국', 0, <Question: 가고 싶은 나라를 선택하세요>, 3)

In [27]:
sc = SubChoice.objects.all().order_by('sub_choice_id')
for c in sc:
    print(c.sub_id, c.sub_choice_id, c.sub_choice, c.description)

1 1 검정색 검은색이다
2 2 파랑색 파랑색이다
3 3 빨강색 빨강색이다
4 4 한국 우리나라다
5 5 미국 가고싶은나라다
6 6 영국 신분세탁국가다
7 7 강아지 강아지는 귀엽다
8 8 고양이 고양이는 귀엽다
9 9 거북이 거북이는 느리다
10 10 러시아 보드카의나라다
11 11 스페인 놀러가고싶은나라다


In [15]:
print(sc.query)

SELECT "polls_subchoice"."sub_id", "polls_subchoice"."sub_choice_id", "polls_subchoice"."description" FROM "polls_subchoice"


In [35]:
for c,s in zip(qs,sc):
    if c.question_id==1:
        print(c.id, c.choice_text, c.vote, c.question, c.question_id, s.sub_id, s.sub_choice, s.description, sep='\t')

1	검정색	0	좋아하는 색깔은 무엇입니까?	1	1	검정색	검은색이다
2	파랑색	0	좋아하는 색깔은 무엇입니까?	1	2	파랑색	파랑색이다
3	빨강색	0	좋아하는 색깔은 무엇입니까?	1	3	빨강색	빨강색이다


# where 절
- filter(조회조건)
    - 조회조건을 만족하는 0개 이상의 행을 조회
- exclude(조회조건)
    - filter의 반대. 조회조건을 만족하지 않는 0개 이상의 행을 조회(not)
- get(조회조건)
    - 조회조건을 만족하는 1개 행을 조회
- filter/exclude : QuerySet으로 반환
- get: Model 객체를 반환

In [50]:
# where id = 1
qs = Question.objects.filter(id=2)
# primary key 컬럼 조회: 컬럼명, pk
qs1 = Question.objects.filter(pk=2)
qs, qs1

(<QuerySet [<Question: 좋아하는 동물은 무엇입니까?>]>,
 <QuerySet [<Question: 좋아하는 동물은 무엇입니까?>]>)

In [51]:
print(qs.first().pub_date)

2021-05-25 05:20:01.945546+00:00


In [53]:
quest = Question.objects.get(pk=2) # get(): pk로 조회
print(type(quest), quest)

<class 'polls.models.Question'> 좋아하는 동물은 무엇입니까?


In [56]:
try:
    r = Question.objects.get(pk=1)
    print(r.pub_date)
except:
    print('조회결과가 없습니다')

2021-05-25 05:18:55.578487+00:00


In [58]:
Question.objects.get(id__gt=1) # id >1

MultipleObjectsReturned: get() returned more than one Question -- it returned 2!

In [59]:
try:
    Question.objects.get(id__gt=1) # id >1
except:
    print('조회결과가 없습니다')

조회결과가 없습니다


In [67]:
# 조회 조건
# qs = Choice.objects.filter(id__gt=7) # id > 7
# qs = Choice.objects.filter(id__gte=7)
qs = Choice.objects.filter(id__lt=5)
# qs = Choice.objects.filter(id__lte=5)
for q in qs:
    print(q.id, q.choice_text)

1 검정색
2 파랑색
3 빨강색
4 한국


In [78]:
qs = Question.objects.filter(question_text__startswith = '가고') # q_t like '가고%'
qs = Question.objects.filter(question_text__endswith = '하세요') # q_t like '%하세요'
qs = Question.objects.filter(question_text__contains = '하는') # q_t like '%하는%'
for q in qs:
    print(q.id, q.question_text)

1 좋아하는 색깔은 무엇입니까?
2 좋아하는 동물은 무엇입니까?


In [82]:
qs = Choice.objects.filter(id__in = [2,5,7,9,20]) # where id in [2,5,7,9,20]
qs = Choice.objects.filter(id__range=(3,7)) # between 3 and 7
print(qs.query)
for q in qs:
    print(q.id, q.choice_text)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."vote", "polls_choice"."question_id" FROM "polls_choice" WHERE "polls_choice"."id" BETWEEN 3 AND 7
3 빨강색
4 한국
5 미국
6 영국
7 강아지


### and 조건

In [86]:
qs = Choice.objects.filter(pk=3, choice_text='빨강색')
qs.first().id, qs.first().choice_text

(3, '빨강색')

In [88]:
qs = Choice.objects.filter(pk__lt=5, choice_text__contains='색')
for q in qs:
    print(q.id, q.choice_text)

1 검정색
2 파랑색
3 빨강색


In [93]:
# OR조건 Q(), ~Q()
from django.db.models import Q
# qs = Choice.objects.filter(Q(pk__gt=5)|Q(choice_text__contains='색'))
# qs = Choice.objects.filter(Q(pk__gt=5)|~Q(choice_text__contains='색')) # Not
qs = Choice.objects.filter(~Q(pk=5))
print(qs.query)
for q in qs:
    print(q.id, q.choice_text)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."vote", "polls_choice"."question_id" FROM "polls_choice" WHERE NOT ("polls_choice"."id" = 5)
1 검정색
2 파랑색
3 빨강색
4 한국
6 영국
7 강아지
8 고양이
9 거북이
10 러시아
11 스페인


In [95]:
qs = Choice.objects.exclude(pk=5)
qs = Choice.objects.exclude(pk__lt=5)
print(qs.query)
for q in qs:
    print(q.id, q.choice_text)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."vote", "polls_choice"."question_id" FROM "polls_choice" WHERE NOT ("polls_choice"."id" < 5)
5 미국
6 영국
7 강아지
8 고양이
9 거북이
10 러시아
11 스페인


# 컬럼 선택
select 컬럼, 컬럼
- values(field이름,....)
    - 개별 조회결과를 Dictionary에 넣어서 반환.

In [103]:
qs = Choice.objects.all().values('id', 'choice_text').order_by('-id')
print(qs.query)
for q in qs:
    print(q['id'], q['choice_text'])

SELECT "polls_choice"."id", "polls_choice"."choice_text" FROM "polls_choice" ORDER BY "polls_choice"."id" DESC
11 스페인
10 러시아
9 거북이
8 고양이
7 강아지
6 영국
5 미국
4 한국
3 빨강색
2 파랑색
1 검정색
